In [ ]:
import csv
from pandas import Series, DataFrame
import pandas as pd
import requests
import json
from os import name
import xml.etree.ElementTree as ET
import pandas as pd
from bs4 import BeautifulSoup
from lxml import html
from urllib.parse import urlencode, quote_plus, unquote
import re
import numpy as np
import pandas as pd
import time
import pickle
import gzip
import pprint
from datetime import datetime
from datetime import timedelta
##API Key
serviceKey = 'Enter your API KEY'

In [ ]:
######Part 1-1 현재일자 기준 인천출발 미래 정기 운항 스케줄 호출(국내선, 국제선 포함)
import requests
from bs4 import BeautifulSoup
data=[]
url = 'https://apis.data.go.kr/B551177/StatusOfPaxFltSched/getPaxFltSchedDeparturesDeOdp'
params ={'serviceKey' : serviceKey, 'numOfRows' : 100 }
response = requests.get(url, params=params)
soup_inc = BeautifulSoup(response.text, 'html.parser')
data = []
for item in soup_inc.find_all('item'):
    item_data = {}
    for tag in item.find_all():
        item_data[tag.name] = tag.text
    data.append(item_data)

df_inc_out= pd.DataFrame(data)
##df_inc.head()


In [ ]:
###airport 체계가 한국공항공사와 달라서 확인 필요함 (ex: 보홀 팡라오)

array(['나트랑', '도쿄/나리타', '오사카/간사이', '세부', '키타큐슈', '보홀 팡라오', '칭다오', '후쿠오카',
       '마닐라', '신 울란바토르', '다낭', '구마모토', '베이징', '하노이', '오키나와', '선전', '홍콩',
       '부산', '상하이/푸동', '삿포로', '다가마쓰', '오카야마', '하얼빈', '선양(심양)', '프랑크푸르트',
       '장사', '광저우', '타이베이', '싱가포르', '옌지(연길)', '호찌민', '방콕/수완나폼', '우한', '괌',
       '시안', '보스턴', '센다이', '애틀랜타', '자무쓰', '뉴욕/존에프케네디', '샤먼', '사이판', '톈진',
       '워싱턴', '옌타이', '토론토', '난징', '웨이하이', '파리/샤를드골', '나고야', '시카고/오헤어',
       '런던/히드로', '가고시마', '암스테르담', '다롄(대련)', '로마', '덴파사르', '지난', '비엔나',
       '델리', '밀라노', '가오슝', '항저우', '두바이', '마드리드', '사가', '마쓰야마', '창춘',
       '요나고', '로스앤젤레스', '카트만두', '이스탄불', '치앙마이', '텐푸국제공항', '자카르타/수카르노하타',
       '쿤밍', '타슈켄트', '충칭', '샌프란시스코', '시애틀', '시즈오카', '정저우', '양곤', '미야자키',
       '쿠알라룸푸르', '양저우', '푸켓', '알마티', '아부다비', '오클랜드(뉴질랜드)', '코타키나발루',
       '밴쿠버', '비엔티안', '시드니', '프놈펜', '칼리보', '대구', '미니애폴리스', '브리즈번', '호놀룰루',
       '두옹 당(푸꿕)', '도쿄/하네다', '클라크필드', '뉴어크', '싼야', '마카오', '헬싱키', '도하',
       '볼레(아디스아바바)', '달랏', '콜롬보', '하이퐁', '히로시마', '부다페스트', '

In [ ]:
######Part 1-2 현재일자 기준 인천 출발하는 항공편
##미래 데이터 적재를 위한 loop
#현재 날짜 가져옴
##today = datetime.today()
today = datetime.today()
data_inc_to_load = []

#미래 30일 데이터
for _ in range(30):
    for index, row in df_inc.iterrows():
        # 날짜 포맷을 고려해 날짜 데이터를 가져옴
        start_date = datetime.strptime(row['firstdate'], '%Y%m%d')
        end_date = datetime.strptime(row['lastdate'], '%Y%m%d')

        # 오늘의 요일을 가져와 대응하는 열을 확인
        today_weekday = today.strftime('%A').lower()

        # 오늘이 시작일자와 종료일자 사이에 있고 해당 요일에 운행하는 경우 데이터를 삽입
        if start_date <= today <= end_date and row[today_weekday] == 'Y':
            row['date'] = today.date()
            data_inc_to_load.append(row)

    # 다음 날짜로 이동
    today += timedelta(days=1)

# 적재된 데이터 출력
df_inc_out = pd.DataFrame(data_inc_to_load)
##result_df.to_csv('test.csv', header=True, encoding='cp949')
df_inc_out.head()


,airline,airport,airportcode,codeshare,firstdate,flightid,friday,lastdate,masterflightid,monday,saturday,season,st,sunday,thursday,tuesday,wednesday,date
0,베트남항공,나트랑,CXR,Master,20231029,VN441,Y,20240330,,Y,Y,W23,0615,Y,Y,Y,Y,2023-11-08
1,대한항공,나트랑,CXR,Slave,20231029,KE5679,Y,20240330,VN441,Y,Y,W23,0615,Y,Y,Y,Y,2023-11-08
2,제주항공,도쿄/나리타,NRT,Master,20231029,7C1100,Y,20231109,,Y,Y,W23,0620,Y,Y,Y,Y,2023-11-08
3,한 에어,도쿄/나리타,NRT,Slave,20231029,H19952,Y,20231109,7C1100,Y,Y,W23,0620,Y,Y,Y,Y,2023-11-08
4,제주항공,오사카/간사이,KIX,Master,20231029,7C1304,Y,20240330,,Y,Y,W23,0700,Y,Y,Y,Y,2023-11-08


In [ ]:
#####Part 1-3 데이터 프레임 전처리 및 현재 테이블 레이아웃에 맞게 변형하기 위한 함수 정의 및 국가/대륙 붙이기 위한 파일 import
##공항과 도시명 변환하기 위한 함수
def split_city_and_airport(row) :
    ##city_parts = row['city'].split('/')
    airport_parts = row['airport'].split('/')
    ##if len(city_parts)>1 :
    ##    row['city'] = city_parts[1]
    if len(airport_parts)>1 :
        row['airport'] = airport_parts[1]
    return row


country = pd.read_csv('airport-codes_csv.csv', keep_default_na=False)
country = country[(country.iata_code.notnull()) & (country.type != 'closed') & (country.type != 'heliport')].loc[:,['continent', 'iso_country', 'iata_code']]
country = country.drop_duplicates(keep='first').loc[country.drop_duplicates(keep='first').iata_code.drop_duplicates(keep=False).index, :]

In [ ]:
#####Part 1-4 해외로 출국하는 Outbound 편(target : tb_ex_flght_arvplc)
#해외로 출국하는 Outbound 항공편이기 때문에 도착지(citycode) 기준으로 join
df_inc_out = pd.merge(df_inc_out, country, how = 'left', left_on='airportcode', right_on='iata_code' )

##Outbound는 airport, airportcode가 출발지, city, citycode가 도착지
df_inc_out = df_inc_out.apply(split_city_and_airport, axis=1)  #공항과 도착지 변환 함수 적용

df_inc_out_fnl = pd.DataFrame()

##미래 데이터 API에서는 estimated_time 필드와 actual_time 필드는 그냥 :로 처리해 줄 수 밖에 없음
df_inc_out_fnl['airline'] = df_inc_out['airline']
df_inc_out_fnl['flight_no'] = df_inc_out['flightid']
df_inc_out_fnl['destination'] = df_inc_out['airportcode'] + '('  + df_inc_out['airport'] + ')'
df_inc_out_fnl['planned_time'] = df_inc_out['st'].str[:2] + ':' + df_inc_out['st'].str[2:]      ##hhmm 으로 있어서 hh:mm으로 변환 필요
df_inc_out_fnl['estimated_time'] = ':'
df_inc_out_fnl['actual_time'] = ':'
df_inc_out_fnl['type']  = '여객'   ###화물 확인하고 추가 변환 해줘야함!!!! 인천 데이터에는 화물 여객 구분 없음
df_inc_out_fnl['status'] = '출발예정'
df_inc_out_fnl['search_date'] = df_inc_out['date']
df_inc_out_fnl['airport'] = '인천'
df_inc_out_fnl['continent'] = df_inc_out['continent']
df_inc_out_fnl['iso_country'] = df_inc_out['iso_country']
df_inc_out_fnl['etl_tm'] = datetime.now().strftime('%Y-%m-%d %H:%M:%S')


########### s3에 떨구고 redshift tb_ex_flght_arvplc 테이블에 적재하는 로직만 쓰면됨


df_inc_out_fnl.head()

,airline,flight_no,destination,planned_time,estimated_time,actual_time,type,status,search_date,airport,continent,iso_country,etl_tm
0,베트남항공,VN441,CXR(나트랑),06:15,:,:,여객,출발예정,2023-11-08,인천,AS,VN,2023-11-08 15:50:00
1,대한항공,KE5679,CXR(나트랑),06:15,:,:,여객,출발예정,2023-11-08,인천,AS,VN,2023-11-08 15:50:00
2,제주항공,7C1100,NRT(나리타),06:20,:,:,여객,출발예정,2023-11-08,인천,AS,JP,2023-11-08 15:50:00
3,한 에어,H19952,NRT(나리타),06:20,:,:,여객,출발예정,2023-11-08,인천,AS,JP,2023-11-08 15:50:00
4,제주항공,7C1304,KIX(간사이),07:00,:,:,여객,출발예정,2023-11-08,인천,AS,JP,2023-11-08 15:50:00
